First we calculate the remainder mod 250 of all terms, putting them into buckets. Bruteforce power mod (even Python's `pow`) is fast enough here, but we may use one trick just to feel good: it's easy to prove a slightly extended version of Euler's theorem here with the help of Chinese remainder theorem:

$$a^103 \equiv a^3 \pmod{250}$$

for all $a$. Then it's very easy to show that if $b \equiv a \pmod 500$, Then

$$b^b \equiv a^a \pmod{250}.$$

So at most we need to calculate the first 500 terms; the rest follows via periodicity.

Now that we felt good and have the number of terms $\equiv r \pmod{250}$ for each $r$, we start from $r = 0$ and build up the subset counts. We let the total number of subsets built from terms $\equiv 0,\,1,\,\dots,\,r \pmod{250}$ s.t. the subset sum $\equiv r' \pmod{250}$ be $f(r, r')$. We further define $g(s)$ to be the number of subsets built only from terms $\equiv r \pmod{250}$ s.t. the subset sum $\equiv s \pmod{250}$. Apparently $g(s)$ is easily calculable:

$$g(s) = \sum_{0\le k \le n,\ kr \equiv s} \binom{n}{k},$$

where $n$ is the number of $\equiv r$ terms. Also, apparently

$$f(r, r') = \sum_{r'' + s \equiv r'} f(r-1, r'') g(s).$$

for $r > 0$ and

$$f(0, r') = g(r').$$

Therefore $f(r, r')$ can be calculated iteratively for each $r$. Eventually we're only interested in $f(249, 0)$. Don't forget to subtract 1 though since the problem asks for the number of *non-empty* subsets. As always this successfully got me the first time round.

The slightly optimized Python implementation took ~3s.

In [1]:
#!/usr/bin/env python3

import collections
import functools

from gmpy2 import mpz


MODULUS = 10_000_000_000_000_000

factorial = [mpz(1)]


def precalc_factorios(n):
    for i in range(1, n + 1):
        factorial.append(factorial[-1] * i)


precalc_factorios(25025)


@functools.lru_cache(maxsize=None)
def binomial(n, k):
    return int(factorial[n] // factorial[k] // factorial[n - k] % MODULUS)


def calculate_remainders():
    remainders = [pow(i % 250, (i - 1) % 100 + 1, 250) for i in range(1, 501)]
    remainders = remainders * (250250 // 500) + remainders[:250]
    return collections.Counter(remainders)


def count_subsets(remainders):
    last_counts = [1] + [0] * 249
    for r, r_count in sorted(remainders.items()):
        additions = [0] * 250
        # Special treatment for 0 and 125 due to large r_count and easy
        # manual calculations.
        if r == 0:
            additions[0] = pow(2, r_count, MODULUS)
        elif r == 125:
            additions[0] = additions[125] = pow(2, r_count - 1, MODULUS)
        else:
            for k in range(r_count + 1):
                kr = r * k % 250
                additions[kr] = (additions[kr] + binomial(r_count, k)) % MODULUS
        current_counts = [0] * 250
        for last in range(250):
            for this in range(250):
                new = (last + this) % 250
                current_counts[new] = (
                    current_counts[new] + last_counts[last] * additions[this]
                ) % MODULUS
        last_counts = current_counts
    # Do not count the empty subset.
    return last_counts[0] - 1


def main():
    remainders = calculate_remainders()
    print(count_subsets(remainders))


if __name__ == "__main__":
    main()


1425480602091519
